# Item 38: Accept Functions Instead of Classes for Simple Interfaces

In [1]:
# Many of Python's built-in APIs allow us to customize behavior by passing in a function. These hooks, 
# are used by APIs to call back our code while they execute. For example, the list type's sort method 
# takes an optional key argument that's used to determine each index's value for sorting.
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=len)
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


In [3]:
# Say that we want to customize the behavior of the defaultdict class. This data structure allows us to supply
# a function that will be called with no arguments each time a missing key is accesed
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After:', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


Supplying funcitons like `log_missing` makes APIs easy to build and test because it separates side effects from deterministic behavior.

In [4]:
# Say we now want the default value hook passed to defaultdict to count the total  number of keys that were
# missing. One way to do this is by using a stateful closure
def increment_with_report(current, increments):
    added_count = 0
    def missing():
        nonlocal added_count # Stateful closure
        added_count += 1
        return 0
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

Running the above function produces the expected result (2), even though the `defaultdict` has no idea that the missing hook maintains state. Another benefit of accepting simple functions for interfaces is that it's easy to add functionality later by hiding state in a closure:

In [5]:
result, count = increment_with_report(current, increments)
assert count == 2

The problem wit defining a closure with stateful hooks is that it's harder to read than the stateless function example.

In [6]:
# Another approach is to define a small class that encapsulates the state you want to track
class CountMissing:
    def __init__(self):
        self.added = 0

    def missing(self):
        self.added += 1
        return 0

In other languages, we might expect that `defaultdict` would have to be modified to accomodate the `CountMissing` interface. But, in Python, thanks to first-class functions, we can reference the `CountMissing.missing` method directly on an object and pass it to `defaultdict` as the default value hook. It's trivial to have an oject instance's method satisfy a function interface:

In [7]:
counter = CountMissing()
result = defaultdict(counter.missing, current) # Method reference
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

Using a helper class like this to provide the behavior of a stateful closure is clearer than using the `increment_with_report` function, as above. But it is still not very clear what the purpose of the `CountMissing` class is.

To clarify this situtation, Python allows classes to define the `__call__` special method. `__call__` allows an `object` to be called just like a function. These are called *callables*.

In [8]:
class BetterCountMissing():
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0

counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

In [9]:
# Here we use a BetterCountMissing instance as the default value hook for a defaultdict to track the number of
# missing keys that were added
counter = BetterCountMissing()
result = defaultdict(counter, current) # Relies on __call__
for key, amount in increments:
    result[key] += amount
assert counter.added == 2